In [1]:
import numpy as np # Linear Algebra
import pinocchio as pin # Pinocchio library
import proxsuite # QP
import example_robot_data # For robot descriptions

from pinocchio.robot_wrapper import RobotWrapper
from pinocchio.shortcuts import buildModelsFromUrdf
from pinocchio.visualize import MeshcatVisualizer

VISUALIZER = MeshcatVisualizer

In [2]:
# Read URDF model
talos = example_robot_data.load("talos")
model, collision_model, visual_model = talos.model, talos.collision_model, talos.visual_model

# Let's create 4 frames per foot (for the contact points)
def add_foot_frames(model, leg, foot_x_size = 0.221, foot_y_size = 0.134, foot_z = -0.107):
    pJointId = model.getJointId("leg_" + leg + "_6_joint")
    pFrameId = model.getFrameId("leg_" + leg + "_sole_fix_joint")
    # [-foot_x_size / 2., -foot_y_size / 2]
    placement = pin.SE3(np.eye(3), np.array([-foot_x_size / 2., -foot_y_size / 2., foot_z]))
    frame = pin.Frame(leg + "_mm", pJointId, pFrameId, placement, pin.BODY)
    model.addFrame(frame)

    # [-foot_x_size / 2., foot_y_size / 2]
    placement = pin.SE3(np.eye(3), np.array([-foot_x_size / 2., foot_y_size / 2., foot_z]))
    frame = pin.Frame(leg + "_mp", pJointId, pFrameId, placement, pin.BODY)
    model.addFrame(frame)

    # [foot_x_size / 2., foot_y_size / 2]
    placement = pin.SE3(np.eye(3), np.array([foot_x_size / 2., foot_y_size / 2., foot_z]))
    frame = pin.Frame(leg + "_pp", pJointId, pFrameId, placement, pin.BODY)
    model.addFrame(frame)

    # [foot_x_size / 2., -foot_y_size / 2]
    placement = pin.SE3(np.eye(3), np.array([foot_x_size / 2., -foot_y_size / 2., foot_z]))
    frame = pin.Frame(leg + "_pm", pJointId, pFrameId, placement, pin.BODY)
    model.addFrame(frame)

add_foot_frames(model, "left")
add_foot_frames(model, "right")

# Create a RobotWrapper for visualization purposes
robot = RobotWrapper(model, collision_model, visual_model)

# Extract pinocchio model and ata
model = robot.model
data = robot.data

In [3]:
# Visualizer!
robot.setVisualizer(VISUALIZER())
robot.initViewer()
robot.loadViewerModel("pinocchio")

You can open the visualizer by visiting the following URL:
http://127.0.0.1:7019/static/


In [6]:
%%html
<iframe src="http://jupyter.lar.upatras.gr:7019/static/" width="640" height="480"></iframe>

In [7]:
# function to compute FK for all frames/joints
def fk_all(model, data, q, v = None):
    if v is not None:
        pin.forwardKinematics(model, data, q, v) # FK and Forward Velocities
    else:
        pin.forwardKinematics(model, data, q) # FK
    pin.updateFramePlacements(model, data) # Update frames

In [8]:
q = pin.neutral(model)
q[2] = 1.0274

# left arm
q[21] = 0.3
q[22] = 0.4
q[23] = -0.5
q[24] = -1.2

# right arm
q[29] = -0.3
q[30] = -0.4
q[31] = 0.5
q[32] = -1.2

# left leg
q[9] = -0.4
q[10] = 0.8
q[11] = -0.4

# right leg
q[15] = -0.4
q[16] = 0.8
q[17] = -0.4

print(q.T)

# Display robot
robot.display(q)

v = np.random.randn(model.nv) * 0.
print(v.T)

# fk_all(model, data, q)
# for frame, oMf in zip(model.frames, data.oMf):
#     print(("{:<10} : {: .2f} {: .2f} {: .2f}"
#           .format(frame.name, *oMf.translation.T.flat )))
# print("================================")

[ 0.      0.      1.0274  0.      0.      0.      1.      0.      0.
 -0.4     0.8    -0.4     0.      0.      0.     -0.4     0.8    -0.4
  0.      0.      0.      0.3     0.4    -0.5    -1.2     0.      0.
  0.      0.     -0.3    -0.4     0.5    -1.2     0.      0.      0.
  0.      0.      0.    ]
[ 0. -0. -0. -0. -0.  0.  0. -0.  0. -0.  0. -0.  0. -0. -0. -0.  0.  0.
 -0.  0.  0.  0.  0.  0. -0. -0.  0.  0. -0. -0. -0.  0. -0.  0. -0.  0.
  0. -0.]


In [9]:
# Let's make a QP object
N_foot = 2
N_contacts = N_foot * 4 # 4 contact points per foot
qp_dim = model.nv + model.nv + 3 * N_contacts
qp_dim_eq = model.nv # dynamics!
qp_dim_in = 5 * N_contacts # Friction cones constraints! ProxSuite supports double limits: d_min <= Cx <= d_max!
qp = proxsuite.proxqp.dense.QP(qp_dim, qp_dim_eq, qp_dim_in)

contact_frames = ["left_mm", "left_mp", "left_pp", "left_pm", "right_mm", "right_mp", "right_pp", "right_pm"]
contact_frames_ids = [model.getFrameId(f) for f in contact_frames]

target_frames = ["base_link", "head_1_link", "arm_left_7_link", "arm_right_7_link"]
target_weights = [1., 1., 0.5, 0.5] # we care more about the base and the head
target_frames_ids = [model.getFrameId(f) for f in target_frames]

# FK
fk_all(model, data, q)

# Compute target poses: we want to stay in the current pose
target_poses = [data.oMf[target_frames_ids[i]].copy() for i in range(len(target_frames_ids))]

# Move right arm target up in Z-axis
target_poses[-1].translation[2] += 0.1
# Move left arm target in Y-axis
target_poses[-2].translation[1] -= 0.2
# Move head/base targets down in Z-axis
target_poses[0].translation[2] -= 0.1
target_poses[1].translation[2] -= 0.1

target_poses[0].translation[0] -= 0.5
target_poses[1].translation[0] -= 0.5

qp_init = False

prev_sol = np.zeros((qp_dim,))

def qp_inverse_dynamics(prev_sol, model, data, q, v, contact_frames, contact_frames_ids, target_frames, target_frames_ids, target_poses, target_weights, mu = 1., n = np.array([0., 0., 1.]), t1 = np.array([1., 0., 0.]), t2 = np.array([0., 1., 0.])):
    global qp_init
    global qp
    # first we do forward kinematics and forward velocities
    fk_all(model, data, q, v)
    # Then compute all Jacobians needed
    pin.computeJointJacobians(model, data, q)
    # We also need the time derivative of the Jacobians
    pin.computeJointJacobiansTimeVariation(model, data, q, v)
    # Mass Matrix
    pin.crba(model, data, q)
    # Coriolis/Gravity forces
    pin.nonLinearEffects(model, data, q, v)

    # Let's create the A matrix
    A = np.zeros((model.nv, qp_dim))
    # put the mass matrix in the correct position
    A[:model.nv, :model.nv] = np.copy(data.M)
    # selection matrix
    S = np.eye(model.nv)
    S[:6, :6] = 0. # we cannot control the base!
    A[:, model.nv:2*model.nv] = -S
    # contact jacobians
    # for frame, frame_id in zip(contact_frames, contact_frames_ids):
    for i in range(len(contact_frames)):
        frame_id = contact_frames_ids[i]
        J = pin.getFrameJacobian(model, data, frame_id, pin.ReferenceFrame.LOCAL_WORLD_ALIGNED)
        A[:, 2*model.nv + i * 3 : 2*model.nv + (i + 1) * 3] = -J[:3, :].T
    # Let's create the b vector
    b = -np.copy(data.nle)

    # Inequality Constraints (Friction cones)
    C = np.zeros((5 * len(contact_frames), qp_dim))
    dmin = np.zeros((5 * len(contact_frames),))
    dmax = np.zeros((5 * len(contact_frames),))
    for i in range(len(contact_frames)):
        # dot(f, n) > 0
        C[i * 5, 2*model.nv + i * 3 : 2*model.nv + (i + 1) * 3] = n # normal vector (world frame)
        dmin[i * 5] = 1e-6
        dmax[i * 5] = None # no upper limit
        # -μ dot(f, n) <= dot(f, t1) <= μ dot(f, n)
        C[i * 5 + 1, 2*model.nv + i * 3 : 2*model.nv + (i + 1) * 3] = t1 - mu * n
        dmin[i * 5 + 1] = None
        dmax[i * 5 + 1] = 0.
        C[i * 5 + 2, 2*model.nv + i * 3 : 2*model.nv + (i + 1) * 3] = t1 + mu * n
        dmin[i * 5 + 2] = 0.
        dmax[i * 5 + 2] = None
        # -μ dot(f, n) <= dot(f, t2) <= μ dot(f, n)
        C[i * 5 + 3, 2*model.nv + i * 3 : 2*model.nv + (i + 1) * 3] = t2 - mu * n
        dmin[i * 5 + 3] = None
        dmax[i * 5 + 3] = 0.
        C[i * 5 + 4, 2*model.nv + i * 3 : 2*model.nv + (i + 1) * 3] = t2 + mu * n
        dmin[i * 5 + 4] = 0.
        dmax[i * 5 + 4] = None

    # Let's create the tasks. One task per contact point and one task per target_frame
    W = np.zeros((len(target_weights) * 6 + len(contact_frames) * 6, qp_dim))
    t = np.zeros((len(target_weights) * 6 + len(contact_frames) * 6,))

    # Contact frames tasks
    for i in range(len(contact_frames)):
        weight = 10. * np.max(target_weights)
        frame_id = contact_frames_ids[i]
        J = pin.getFrameJacobian(model, data, frame_id, pin.ReferenceFrame.LOCAL)
        dJ = pin.getFrameJacobianTimeVariation(model, data, frame_id, pin.ReferenceFrame.LOCAL)
        W[i * 6 : (i + 1) * 6, : model.nv] = J * weight # update W
        # We do not use a PD-controller here because it can generate jitter!
        t[i * 6 : (i + 1) * 6] = (-dJ @ v) * weight # update t

    # Target frames tasks
    idx = len(contact_frames) * 6
    for i in range(len(target_weights)):
        weight = target_weights[i]
        frame_id = target_frames_ids[i]
        T_wd = target_poses[i]
        # get Jacobian and time derivative of Jacobian
        J = pin.getFrameJacobian(model, data, frame_id, pin.ReferenceFrame.LOCAL)
        dJ = pin.getFrameJacobianTimeVariation(model, data, frame_id, pin.ReferenceFrame.LOCAL)
        W[idx + i * 6 : idx + (i + 1) * 6, : model.nv] = J * weight # update W
        # compute desired acceleration (PD-controller)
        current_pose = data.oMf[frame_id].copy()
        current_vel = pin.getFrameVelocity(model, data, frame_id, pin.ReferenceFrame.LOCAL).vector
        error = pin.log(current_pose.actInv(T_wd)).vector
        Kp = 10.
        Kd = 2. * np.sqrt(Kp)
        p_ddot = Kp * error - Kd * current_vel
        t[idx + i * 6 : idx + (i + 1) * 6] = (p_ddot - dJ @ v) * weight # update t

    # Let's create the big Q matrix and q vector
    # regularization tasks (very important)
    reg = np.eye(qp_dim) * 1e-6
    reg[:model.nv,:model.nv] = np.eye(model.nv) * 1e-2 # We want to minimize accelerations as much as possible

    Q = W.T @ W + reg
    q_o = -W.T @ t

    # Let's solve the QP
    if not qp_init: # in first iteration we initialize the model
        qp.init(Q, q_o, A, b, C, dmin, dmax)
    else: # otherwise, we update the model
        qp.update(Q, q_o, A, b, C, dmin, dmax)
    # Let's solve the QP
    qp.solve()
    qp_init = True
    prev_sol = np.copy(qp.results.x)
    # We get back the results
    return np.copy(qp.results.x)

In [10]:
def decode_qp_sol(sol):
    dq = sol[:model.nv]
    tau = sol[model.nv:2*model.nv]
    F_left = sol[2*model.nv:2*model.nv + 4 * 3]
    F_right = sol[2*model.nv + 4 * 3:]
    return dq, tau, F_left, F_right

In [11]:
sol = qp_inverse_dynamics(prev_sol, model, data, q, v, contact_frames, contact_frames_ids, target_frames, target_frames_ids, target_poses, target_weights)

dq, tau, F_left, F_right = decode_qp_sol(sol)

np.set_printoptions(precision=3, suppress=True, threshold=10000, linewidth=1000)
print("dq:", dq.T)
print("tau:", tau.T)
print("F (left):", F_left.T)
print("F (right):", F_right.T)

dq: [-1.261 -0.052 -0.44  -0.02  -0.374 -0.009  0.009  0.108 -2.8    3.012  0.162 -0.088  0.009  0.107 -2.791  2.987  0.178 -0.088  0.011 -3.82  -7.688 -2.686  1.873  2.867  1.037  3.366  4.684 -0.026  7.491 -0.041 -3.83   1.554 -2.439 -0.613  4.614 -0.024  4.181  0.007]
tau: [  0.      0.      0.      0.      0.      0.      0.556   4.659  20.576 -18.982  47.245  -1.982   1.074  -0.874  13.582 -16.135  39.265  -1.11   -1.29  -15.121  -2.613   0.358   2.452  -8.595   0.      0.718  -1.115   0.015   1.451  -3.426  -4.188  -8.437  -0.215  -0.727  -1.049   0.019  -0.019  -0.   ]
F (left): [ -0.      0.      0.      0.      0.      0.    -19.889  -0.737 247.778 -21.996  -0.737 216.013]
F (right): [ -0.      0.      0.     -0.      0.      0.    -17.921  -4.627 209.421 -19.958  -4.627 178.257]


In [13]:
%%html
<iframe src="http://jupyter.lar.upatras.gr:7019/static/" width="640" height="480"></iframe>

In [ ]:
# Let's do a small simulation loop
T = 10.
dt = 0.005
K = int(T/dt) + 1

for k in range(K):
    # We solve the QP
    sol = qp_inverse_dynamics(prev_sol, model, data, q, v, contact_frames, contact_frames_ids, target_frames, target_frames_ids, target_poses, target_weights)
    # We get the result
    dq, tau, F_left, F_right = decode_qp_sol(sol)
    # Let's do semi-implicit euler integration
    v = v + dq * dt
    q = pin.integrate(model, q, v * dt)
    # Display robot
    if k % 5 == 0:
        robot.display(q)
print("F (left):", F_left.T)
print("F (right):", F_right.T)